In [1]:
import qiskit

In [8]:
# Biased Quantum Galton Box - Full Integrated Code (Ry + Exponential Comparison)

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import matplotlib.pyplot as plt
from scipy.stats import binom
from collections import Counter
import numpy as np
import os

# Layers to run
layers_list = [5, 10, 15, 20, 25]

# Bias angle for exponential-like output
theta = np.pi / 6   # Try π/6 for left-skewed, or π/3 for right-skewed

# Output folder
output_folder = './results_quantum_biased'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Biased Quantum Galton Box function
def run_biased_galton_box(n_layers, theta, shots=10000):
    qc = QuantumCircuit(n_layers, n_layers)

    # Apply Ry(2θ) instead of Hadamard
    for i in range(n_layers):
        qc.ry(2 * theta, i)

    qc.measure(range(n_layers), range(n_layers))

    # Run circuit
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=shots)
    result = job.result()
    counts = result.get_counts()

    # Process counts to number of 1's (right moves)
    position_counts = Counter()
    for bitstring, freq in counts.items():
        num_right = bitstring.count('1')
        position_counts[num_right] += freq

    return qc, position_counts

for n_layers in layers_list:
    print(f"Running Biased Quantum Galton Box: {n_layers} layers...")

    qc, position_counts = run_biased_galton_box(n_layers, theta, shots=10000)

    # Create figure with subplots
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    # Plot circuit diagram
    circuit_drawer(qc, output='mpl', ax=axs[0])
    axs[0].set_title(f'Biased Quantum Circuit\n{n_layers} Layers, θ={round(theta,2)}')

    # Histogram of Quantum Simulation
    total_counts = sum(position_counts.values())
    quantum_probs = [position_counts.get(i, 0) / total_counts for i in range(n_layers+1)]

    axs[1].bar(range(n_layers+1), quantum_probs, color='salmon', alpha=0.7, label='Quantum Simulation')
    axs[1].set_xlabel('Number of "Right" Moves (1\'s)')
    axs[1].set_ylabel('Probability')
    axs[1].set_title('Quantum Galton Box Output')
    axs[1].legend()

    # Compare to Ideal Exponential Distribution
    lmbda = 1.0  # You can adjust for better visual fit
    x = np.arange(0, n_layers+1)
    exp_probs = np.exp(-lmbda * x)
    exp_probs /= np.sum(exp_probs)  # Normalize

    axs[2].bar(range(n_layers+1), quantum_probs, color='salmon', alpha=0.5, label='Quantum Simulation')
    axs[2].plot(x, exp_probs, 'bo-', label='Ideal Exponential', markersize=8)
    axs[2].set_xlabel('Number of "Right" Moves (1\'s)')
    axs[2].set_ylabel('Probability')
    axs[2].set_title('Quantum vs Exponential')
    axs[2].legend()

    plt.tight_layout()

    # Save the figure
    filename = f"{output_folder}/biased_quantum_galton_box_{n_layers}_layers.png"
    plt.savefig(filename)
    plt.close()

    print(f"Saved result: {filename}")


Running Biased Quantum Galton Box: 5 layers...
Saved result: ./results_quantum_biased/biased_quantum_galton_box_5_layers.png
Running Biased Quantum Galton Box: 10 layers...
Saved result: ./results_quantum_biased/biased_quantum_galton_box_10_layers.png
Running Biased Quantum Galton Box: 15 layers...
Saved result: ./results_quantum_biased/biased_quantum_galton_box_15_layers.png
Running Biased Quantum Galton Box: 20 layers...
Saved result: ./results_quantum_biased/biased_quantum_galton_box_20_layers.png
Running Biased Quantum Galton Box: 25 layers...
Saved result: ./results_quantum_biased/biased_quantum_galton_box_25_layers.png
